<a href="https://colab.research.google.com/github/tazar09/cars_dimensions/blob/main/midsize_16-feb2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import numpy as np

import warnings
warnings.filterwarnings('ignore')
import time

# Scrapping the website

In [173]:
# Creating the list of car categories

def make_cat_list(url = 'https://www.automobiledimension.com/large-suv-4x4-cars.php'):
  peq = requests.get(url_car_categories, verify = False)
  peq_content = peq.text
  peq_soup = BeautifulSoup(peq_content, 'html.parser')
  main_peq = peq_soup.find_all('section')
  main_peq = main_peq[-1]

  category_list = []
  for i in main_peq.find_all('a'):
   link = str(i['href'])
   category_list.append(link)
  return category_list

In [177]:
category_list = make_cat_list()
category_list

['city-cars.php',
 'small-cars.php',
 'compact-cars.php',
 'family-cars.php',
 'executive-cars.php',
 'luxury-cars.php',
 'sports-cars.php',
 'estate-cars.php',
 'mpv-cars.php',
 'small-suv.php',
 'compact-suv.php',
 'mid-size-suv.php',
 'pick-up-vehicles.php',
 'passenger-vans.php']

In [178]:
# Creating the list of car from a category

def scrap_site(category_list = category_list):
  carlist = []
  for i in category_list:
    url = 'https://www.automobiledimension.com/' + i

    time.sleep(5)

    h3 = requests.get(url, verify = False)
    html_content = h3.text
    soup = BeautifulSoup(html_content, 'html.parser')
    main_table = soup.find('main')

    car_list_one_cat = []
    for i in main_table.find_all('div', class_ = 'unit'):
      link = str(i.a['href'])
      car_list_one_cat.append('https://www.automobiledimension.com' + link)

    carlist.extend(car_list_one_cat)

  return carlist

In [ ]:
lst = scrap_site()

In [182]:
len(lst)

360

In [2]:
# url = 'https://www.automobiledimension.com/mid-size-suv.php'
# h3 = requests.get(url, verify = False)
# html_content = h3.text

# Creating BeautifulSoup objects

In [3]:
# soup = BeautifulSoup(html_content, 'html.parser')
# main_table = soup.find('main')

In [4]:
# temp_list = []
# for i in main_table.find_all('div', class_ = 'unit'):
#   link = str(i.a['href'])
#   temp_list.append('https://www.automobiledimension.com' + link)
# temp_list[0:3]

['https://www.automobiledimension.com/model/hyundai/kona',
 'https://www.automobiledimension.com/model/renault/austral',
 'https://www.automobiledimension.com/model/kia/sportage']

# Extract model description

In [185]:
def extract_main_text(text):
  search = text.find('div', class_ = 'interior-text')
  return search.text

In [186]:
def extract_boot_space(text):
  search = text.find_all('figcaption')[1]
  boot_space = search.text
  return boot_space

In [187]:
main_text_list = []
boot_space_list = []

try:
  for i in lst:
    content = requests.get(i, verify = False)
    time.sleep(5)
    content = BeautifulSoup(content.text, 'html.parser')
    main_text_list.append(extract_main_text(content))
    boot_space_list.append(extract_boot_space(content))
except:
  main_text_list.append(None)
  boot_space_list.append(None)

# Creating dataframe

In [216]:
df = pd.DataFrame({'description': main_text_list, 'boot_space': boot_space_list})
df.head(360)

,description,boot_space
0,"The Citroen Ami has a length of 2410 mm, a hei...",Boot space: 63 liters.
1,"The Mobilize Duo has a length of 2430 mm, a he...",Boot space: liters.
2,"The Fiat Topolino has a length of 2530 mm, a h...",Boot space: 63 liters.
3,"The Seat Mii electric has a length of 3556 mm,...",Boot space: 251 liters.
4,"The Fiat 500 has a length of 3632 mm, a height...",Boot space: 185 liters.
...,...,...
355,The Peugeot Traveller Long has a length of 530...,Boot space: (6) 2932 - (9) 989 liters.
356,The Toyota Proace Verso Long has a length of 5...,Boot space: (6) 2932 - (9) 989 liters.
357,The Mercedes-Benz V xlg has a length of 5370 m...,Boot space: (5) 3200 - (8) 1410 liters.
358,The Renault Grand Trafic Combi has a length of...,Boot space: (6) 3400 - (9) 1800 liters.


In [217]:
df.to_csv('df.csv')

# Cleaning the dataframe

In [215]:
df['boot_space'] = df['boot_space'].str.replace('Boot space: ', '')
df['boot_space'] = df['boot_space'].str.replace(' liters.','')
df['boot_space'] = df['boot_space'].str.replace('*','')
df.head(360)

KeyError: 'boot_space'

In [199]:
x = df.loc[31, 'boot_space']
len(x)

3

In [200]:
def extract_5_seats_boot(text):
  # rows with only 3-digits number
  if len(text) == 3:
    return text
  # rows w/wo hybrid
  elif ('-' in text and '(5)' not in text):
    pattern1 = r'.*(\d{3})'
    match1 = re.search(pattern1, text)
    return match1.group(1).strip()
  # rows with 5 and 7 seated boot space
  elif '(5)' in text:
    pattern2 = r'\(5\)\s(\d+)'
    match2 = re.search(pattern2, text)
    return match2.group(1).strip()
  # blank and all other rows
  else:
    return ''

def extract_7_seats_boot(text):
  if '(7) ' in text:
    pattern = r'\(7\)\s(\d+)'
    match1 = re.search(pattern, text)
    if match1:
      return match1.group(1).strip()
    else:
      return ''
  else:
    return ''

def extract_5_hybrid(text):
    if ('-' in text and '(' not in text):
      first, second = text.split('-')
      first = first.strip()
      second = second.strip()
      lst1 = [first, second]
      lst1.sort()
      return lst1[0]


    elif ('(' in text and '-' in text):
       pattern = r".*(\d{3}).*(\d{3}).*(\d{3})"
       match1 = re.search(pattern, text)
       if match1:
        first = int(match1.group(1))
        second = int(match1.group(2))
        third = int(match1.group(3))
        lst2 = [first, second, third]
        lst2.sort()
        return lst2[1]
       else:
        return ''
    else:
      return ''

In [214]:
df['boot_space']

KeyError: 'boot_space'

# here you left

In [201]:
# df['boot_space_5'] = df['boot_space'].apply(lambda x: extract_5_seats_boot(x))
df['boot_5_hybrid'] = df['boot_space'].apply(lambda x: extract_5_hybrid(x))
df['boot_space_7'] = df['boot_space'].apply(lambda x: extract_7_seats_boot(x))

# df.drop('boot_space', axis = 1).head(100)

In [ ]:
# df['7-seated'] = df['boot_space_7'].apply(lambda x: 'Yes' if pd.notna(x) else 'No')

# def boot_hybrid(text):
#   if ('-' in text and '(' not in text):
#     match_hybrid = text.split('-')
#     return match_hybrid[0].strip()
#   else:
#     return np.NaN

# def boot_normal(text):
#   if ('-' in text and '(' not in text):
#     match_normal = text.split('-')
#     return match_normal[1].strip()
#   else:
#     return np.NaN

# # def join_boot_5_normal(text)



# df['boot_space_hydbrid'] = df['boot_space'].apply(lambda x: boot_hybrid(x))
# df['boot_space_normal'] = df['boot_space'].apply(lambda x: boot_normal(x))
# df['boot_normal_final'] = df['boot_space_normal'].apply(lambda x: fillna())
# df.head(100)

# Extract dimensions

In [202]:
def extract_dim(text):
  patterns = {
  'name': r'The\s+(.*?)\s+has',
  "length": r"length\s+of\s+(\d+)\s+mm",
  'measurement': r'measurement\s+of\s+(\d+)\s+millimeters',
  "height": r"height\s+of\s+(\d+)\s+mm",
  "width": r"width\s+of\s+(\d+)\s+mm",
  "ground_clearance": r"ground\s+clearance\s+of\s+(\d+)\s+cm",
  "category": r"category\s+of\s+(.*?)\."
  }

  car_info = {}

  for key, pattern in patterns.items():
    match = re.search(pattern, text)
    if match:
      car_info[key] = match.group(1) # Convert matched value to integer
    else:
      car_info[key] = None  # Set missing values to None

  return list(car_info.values()), list(car_info.keys())

In [203]:
columns = extract_dim(df['description'][0])[1]
for i in columns:
  df[i] = df['description'].apply(lambda x: extract_dim(x)[0][columns.index(i)])
# df.drop(['boot_space', 'description'], axis = 1).head(10)

In [204]:
df['ground_clearance'] = df['ground_clearance'].replace('None', '')

In [206]:
pattern = r"^(.*?)\s+(.*)$"
df[['brand', 'model']] = df['name'].str.extract(pattern, expand=True)

df = df[['brand', 'model', 'category', 'boot_5_hybrid',
       'boot_space_7', 'length', 'measurement', 'height', 'width',
       'ground_clearance']]

# df.head(10)

In [207]:
df['ground_clearance'] = df['ground_clearance'].fillna('')
df['measurement'] = df['measurement'].fillna('')
df.head(10)

,brand,model,category,boot_5_hybrid,boot_space_7,length,measurement,height,width,ground_clearance
0,Citroen,Ami,city cars,,,2410,,1520,1390,
1,Mobilize,Duo,city cars,,,2430,,1460,1300,
2,Fiat,Topolino,city cars,,,2530,,1520,1390,
3,Seat,Mii electric,city cars,,,3556,1910,1481,1645,
4,Fiat,500,city cars,,,3632,1900,1527,1683,
5,Kia,Picanto,city cars,,,3595,,1485,1595,
6,Volkswagen,up!,city cars,,,3600,1910,1504,1645,
7,Renault,Twingo,city cars,,,3615,,1541,1646,
8,Fiat,500,city cars,,,3632,1900,1527,1683,
9,Hyundai,i10,city cars,,,3670,,1480,1680,


In [208]:
for i in df.columns[3:]:
  df[i] = df[i].replace('', np.NaN)
  df[i] = df[i].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   brand             360 non-null    object 
 1   model             360 non-null    object 
 2   category          360 non-null    object 
 3   boot_5_hybrid     87 non-null     float64
 4   boot_space_7      15 non-null     float64
 5   length            360 non-null    float64
 6   measurement       256 non-null    float64
 7   height            360 non-null    float64
 8   width             360 non-null    float64
 9   ground_clearance  96 non-null     float64
dtypes: float64(7), object(3)
memory usage: 28.2+ KB


In [209]:
df.isnull().sum()

brand                 0
model                 0
category              0
boot_5_hybrid       273
boot_space_7        345
length                0
measurement         104
height                0
width                 0
ground_clearance    264
dtype: int64

In [210]:
df['brand'].value_counts()

Volkswagen       22
Mercedes-Benz    21
Toyota           20
Audi             19
BMW              16
Ford             16
Peugeot          15
Hyundai          14
Opel             13
Citroen          13
Renault          13
Kia              12
Fiat             11
Mazda             9
Skoda             9
Nissan            8
Honda             7
Lexus             7
Jaguar            7
Suzuki            6
Seat              6
Porsche           6
CUPRA             6
Volvo             6
Mitsubishi        5
Ferrari           5
Subaru            5
Dacia             5
MINI              4
DS                4
BYD               4
MG                4
Maserati          4
SsangYong         4
Jeep              4
McLaren           4
Alfa-Romeo        3
Rolls-Royce       3
Aston-Martin      3
Lamborghini       3
Smart             2
Land-Rover        2
Tesla             2
Bentley           2
Chevrolet         1
Alpine            1
Polestar          1
Mobilize          1
LynkCo            1
Isuzu             1


In [211]:
df[(df['brand'] == 'Mercedes-Benz') ^ (df['brand'] == 'BMW')].sort_values('brand')

,brand,model,category,boot_5_hybrid,boot_space_7,length,measurement,height,width,ground_clearance
20,BMW,i3,small cars,NaN,NaN,4011.0,2039.0,1578.0,1775.0,NaN
198,BMW,2 Active Tourer,MPV cars,406.0,NaN,4386.0,2102.0,1576.0,1824.0,NaN
139,BMW,Z4,sports cars,NaN,NaN,4324.0,2024.0,1304.0,1864.0,NaN
123,BMW,8 Gran Coupe,luxury cars,NaN,NaN,5082.0,2137.0,1407.0,1932.0,NaN
117,BMW,8 Coupe,luxury cars,NaN,NaN,4851.0,2137.0,1346.0,1902.0,NaN
116,BMW,5,executive cars,490.0,NaN,5060.0,2156.0,1515.0,1900.0,NaN
182,BMW,3 Touring,estate cars,410.0,NaN,4713.0,2068.0,1440.0,1827.0,NaN
92,BMW,i4,executive cars,NaN,NaN,4783.0,2081.0,1448.0,1852.0,NaN
91,BMW,4 Coupe,executive cars,NaN,NaN,4768.0,2081.0,1383.0,1852.0,NaN
274,BMW,X1,compact SUV,490.0,NaN,4500.0,2104.0,1642.0,1845.0,20.0
